In [4]:
from pysolar.solar import get_altitude, get_azimuth
from datetime import datetime, timedelta
from pytz import timezone
import numpy as np

def calculate_solar_positions(
    start_date: datetime,
    start_hour: int = 10,
    end_hour: int = 17,
    latitude: float = -0.2105367,
    longitude: float = -78.491614
):
    """Calcula posiciones solares y ángulos para una fecha específica y rango de horas.

    ## Parameters
    - start_date: Fecha y hora de inicio para la simulación (se usa solo la fecha, hora se define con start_hour).
    - start_hour: Hora de inicio del cálculo.
    - end_hour: Hora de fin del cálculo.
    - latitude: Latitud para la posición geográfica.
    - longitude: Longitud para la posición geográfica.

    ## Return
    - times: Lista de tiempos de simulación.
    - azimuths: Lista de ángulos azimutales.
    - elevations: Lista de ángulos de elevación.
    - beta: Lista de ángulos de rollo calculados.
    - phi: Lista de ángulos de inclinación calculados.
    """
    times = []
    azimuths = []
    elevations = []
    beta = []
    phi = []

    time_interval = timedelta(hours=0.5)

    # Crear el rango de tiempos basado en la hora de inicio y fin
    start_time = start_date.replace(hour=start_hour, minute=0, second=0, microsecond=0)
    end_time = start_date.replace(hour=end_hour, minute=0, second=0, microsecond=0)

    current_time = start_time
    while current_time <= end_time:
        az = get_azimuth(latitude, longitude, current_time)
        el = get_altitude(latitude, longitude, current_time)

        # Convertir a radianes para los cálculos
        az_rad = (az * np.pi) / 180
        el_rad = (el * np.pi) / 180

        val = np.cos(el_rad) * np.sin(az_rad)
        beta_rad = np.arcsin(val)
        beta_deg = (beta_rad * 180) / np.pi

        val_fi1 = -(np.cos(el_rad) * np.cos(az_rad)) / np.cos(beta_rad)
        x_rad = np.arcsin(val_fi1)
        x_deg = (x_rad * 180) / np.pi

        val_fi2 = np.sin(el_rad) / np.cos(beta_rad)
        y_rad = np.arccos(val_fi2)
        y_deg = (y_rad * 180) / np.pi

        times.append(current_time)
        azimuths.append(az)
        elevations.append(el)
        beta.append(beta_deg)
        phi.append(x_deg)  # Asumiendo que phi corresponde a x_deg; ajusta si es necesario

        current_time += time_interval

    return times, azimuths, elevations, beta, phi

In [5]:
start_date = datetime(int("2024"), int("6"), int("20"), tzinfo=timezone("America/Guayaquil"))
times, azimuths, elevations, beta, phi = calculate_solar_positions(start_date, start_hour=int("7"), end_hour=int("15"))


In [6]:
# Supongamos que ya has calculado los valores de times, azimuths, elevations, beta, y phi
# Aquí está el código para imprimirlos en un formato legible.

print("Solar Position Data:\n")

print("Times: ", end="")
print(", ".join([f"{time.strftime('%H:%M')}" for time in times]))

print("\nAzimuths: ", end="")
print(", ".join([f"{azimuth:.2f}°" for azimuth in azimuths]))

print("\nElevations: ", end="")
print(", ".join([f"{elevation:.2f}°" for elevation in elevations]))

print("\nBeta (Ángulo de Inclinación): ", end="")
print(", ".join([f"{b:.2f}°" for b in beta]))

print("\nPhi (Ángulo de Orientación): ", end="")
print(", ".join([f"{p:.2f}°" for p in phi]))

Solar Position Data:

Times: 07:00, 07:30, 08:00, 08:30, 09:00, 09:30, 10:00, 10:30, 11:00, 11:30, 12:00, 12:30, 13:00, 13:30, 14:00, 14:30, 15:00

Azimuths: 65.69°, 64.65°, 63.11°, 60.94°, 57.95°, 53.84°, 48.14°, 40.20°, 29.25°, 14.88°, 358.11°, 341.67°, 328.04°, 317.81°, 310.44°, 305.14°, 301.30°

Elevations: 14.48°, 21.26°, 27.99°, 34.61°, 41.07°, 47.28°, 53.11°, 58.35°, 62.64°, 65.49°, 66.34°, 65.01°, 61.80°, 57.26°, 51.86°, 45.93°, 39.66°

Beta (Ángulo de Inclinación): 61.94°, 57.37°, 51.96°, 46.01°, 39.72°, 33.21°, 26.55°, 19.80°, 12.98°, 6.12°, -0.76°, -7.63°, -14.49°, -21.30°, -28.03°, -34.67°, -41.13°

Phi (Ángulo de Orientación): -57.90°, -47.73°, -40.40°, -35.14°, -31.34°, -28.59°, -26.60°, -25.21°, -24.29°, -23.78°, -23.64°, -23.86°, -24.46°, -25.47°, -26.99°, -29.12°, -32.08°


In [8]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import tkinter as tk
import numpy as np

# Extraer los datos de tiempo, azimuth y elevación
#times = [pos[0] for pos in solar_positions]
#azimuths = [pos[1] for pos in solar_positions]
#elevations = [pos[2] for pos in solar_positions]
#beta = [sol[0] for sol in solucion]
#phi = [sol[1] for sol in solucion]

fecha = input("Ingrese una fecha en formato año-mes-día (ejemplo: 2024-08-11): ")
anio, mes, dia = fecha.split('-')
rango = input("Ingrese el rango de la simulación (ejemplo: 10-17): ")
inicio, fin = rango.split('-')

start_date = datetime(int(anio), int(mes), int(dia), tzinfo=timezone("America/Guayaquil"))
times, azimuths, elevations, beta, phi = calculate_solar_positions(start_date, start_hour=int(inicio), end_hour=int(fin))

In [11]:
# Configurar la ventana de Tkinter
root = tk.Tk()
root.title("Simulación de Ángulos Solares")

# Crear un marco para la gráfica 3D
frame_3d = tk.Frame(root)
frame_3d.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Crear la figura y el eje 3D
fig3 = plt.Figure(figsize=(7, 7), dpi=100)
ax3 = fig3.add_subplot(111, projection='3d')

# Convertir los tiempos a números (por simplicidad en la gráfica 3D)
time_numbers = np.arange(len(times))

# Configurar los ejes
ax3.set_xlabel('Time')
ax3.set_ylabel('Azimuth / Roll (degrees)')
ax3.set_zlabel('Elevation / Pitch (degrees)')
ax3.set_title('Solar and Panel Trajectories')
ax3.legend()

# Inicializar las listas de puntos
points_solar, = ax3.plot([], [], [], 'bo', markersize=5)
points_panel, = ax3.plot([], [], [], 'r^', markersize=5)

# Función de inicialización para la animación 3D
def init_3d():
    ax3.set_xlim(min(time_numbers), max(time_numbers))
    ax3.set_ylim(min(min(azimuths), min(beta)), max(max(azimuths), max(beta)))
    ax3.set_zlim(min(min(elevations), min(phi)), max(max(elevations), max(phi)))
    return points_solar, points_panel

# Función de actualización para la animación 3D
def update_3d(frame):
    xdata = time_numbers[:frame+1]
    ydata_solar = azimuths[:frame+1]
    zdata_solar = elevations[:frame+1]
    ydata_panel = beta[:frame+1]
    zdata_panel = phi[:frame+1]
    points_solar.set_data(xdata, ydata_solar)
    points_solar.set_3d_properties(zdata_solar)
    points_panel.set_data(xdata, ydata_panel)
    points_panel.set_3d_properties(zdata_panel)
    return points_solar, points_panel

# Crear el lienzo para la figura 3D
canvas3 = FigureCanvasTkAgg(fig3, master=frame_3d)
canvas3.draw()
canvas3.get_tk_widget().pack(fill=tk.BOTH, expand=True)

interval_time_ms = 500
ani3 = animation.FuncAnimation(fig3, update_3d, frames=len(times), init_func=init_3d, interval=interval_time_ms, blit=True)

# Iniciar la interfaz gráfica de Tkinter
root.mainloop()

C:\Users\Enrique_P\AppData\Local\Temp\ipykernel_8560\1777517097.py:21: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax3.legend()
